## Tests for interpolation by Chebyshev polynomials in the TT-format with cross approximation

> Tests for interpolation by Chebyshev polynomials in the TT-format with cross approximation

Let consider 4D function of the form

$$
f(x) = 7 + x_1^2 + e^{-x_2} + \sin{x_3} + x_4^3
$$

In [1]:
import sys
import time

import numpy as np
import matplotlib.pyplot as plt

import tt

sys.path.append('./../lib')
from intertrain import Intertrain

In [2]:
def func(x):
    return 7. + x[0, ]**2 + np.exp(-x[1, ]) + np.sin(x[2, ]) + x[3, ]**3

In [12]:
IT = Intertrain(
    n=[20, 20, 20, 20],
    l=[ [-3., 3.], [-3., 3.], [-3., 3.], [-3., 3.] ],
    eps=1.E-6
)

### Interpolation from explicit tensor of function values

In [13]:
Y = np.zeros(IT.n)
for ind, v in np.ndenumerate(Y):
    Y[ind] = func(IT.point(np.array(ind)))
Y = tt.tensor(Y, IT.eps)

IT.init(Y=Y)
IT.prep()
IT.info(func)

------------------ Parameters
Dimensions       :        4
Accuracy         : 1.00e-06
Dim 1  | Poi 20  | Min -3.000 | Max 3.000  |
Dim 2  | Poi 20  | Min -3.000 | Max 3.000  |
Dim 3  | Poi 20  | Min -3.000 | Max 3.000  |
Dim 4  | Poi 20  | Min -3.000 | Max 3.000  |
------------------ Test (random points)
Number of points : 10
Error (max)      : 8.07e-15 
Error (mean)     : 3.26e-15 
Error (min)      : 7.77e-16 
------------------ Time
Init             : 9.54e-07 sec. 
Prep             : 5.14e-04 sec. 
Calc (average)   : 2.21e-04 sec. 
Func (average)   : 0.00e+00 sec. 
------------------


### Interpolation with cross approximation

In [14]:
IT.init(func)
IT.prep()
IT.info()

------------------ Parameters
Dimensions       :        4
Accuracy         : 1.00e-06
Dim 1  | Poi 20  | Min -3.000 | Max 3.000  |
Dim 2  | Poi 20  | Min -3.000 | Max 3.000  |
Dim 3  | Poi 20  | Min -3.000 | Max 3.000  |
Dim 4  | Poi 20  | Min -3.000 | Max 3.000  |
------------------ Test (random points)
Number of points : 10
Error (max)      : 1.78e-12 
Error (mean)     : 1.43e-12 
Error (min)      : 1.08e-12 
------------------ Time
Init             : 8.00e+00 sec. 
Prep             : 4.19e-02 sec. 
Calc (average)   : 3.54e-04 sec. 
Func (average)   : 0.00e+00 sec. 
------------------


In [6]:
20**4

160000

In [15]:
IT.calc_num

137523